# `tensorflow.keras` transfer learning from pretrained weights


This notebook demonstrates the use of (1) a well-known CNN architecture and (2) pretrained weights on `RadImageNet` in a multiclassification task.

References:

* The pretrained weights published by [Mei et al (2022)](https://pubs.rsna.org/doi/10.1148/ryai.210315) in their github [link](https://github.com/BMEII-AI/RadImageNet).

* The github repository form the BAGLS team contained in this github [link](https://github.com/anki-xyz/bagls/blob/master/Utils/DataGenerator.py#L109)


In [1]:
# dev convenience
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
import PATHS

import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["WANDB_SILENT"] = "True"
os.environ["WANDB_NOTEBOOK_NAME"] = "04-tf-pretrained-cat.ipynb"
PROJECT_NAME = 'bagls-sh-test'
RUN_NAME = 'pretrained_from_radimagenet'
METRICS_TABLE_NAME = 'metrics_table'
GRADCAM_LAYER_NAME = "conv5_block3_out"

In [3]:
import wandb
print("W&B: ", wandb.__version__)
wandb.login()

# # manage logs
# import logging

# logger = logging.getLogger("wandb")
# logger.setLevel(logging.ERROR)

# logging.getLogger('tensorflow').disabled = True

W&B:  0.13.5


wandb: Currently logged in as: miked. Use `wandb login --relogin` to force relogin


True

In [4]:
# tf loader
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras

In [5]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6283408773424208427
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2115178917808443360
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1"
]


2022-11-29 09:09:49.967660: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 09:09:50.976638: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1


In [6]:
import config
configs = config.nb_configs

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# initialize data generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=configs["validation_split"],
    rescale=configs["rescale"],
    width_shift_range=configs["width_shift_range"],
    height_shift_range=configs["height_shift_range"],
    shear_range=configs["shear_range"],
    zoom_range=configs["zoom_range"],
    fill_mode=configs["fill_mode"],
    horizontal_flip=configs["horizontal_flip"],
    rotation_range=configs["rotation_range"],
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    rescale=configs["rescale"],
)

In [8]:
train_dir = configs["train_dir"]
test_dir = configs["test_dir"]

batch_size = configs["batch_size"]
class_names = configs["class_names"]
interpol = configs["interpol"]
cmap = configs["cmap"]
label_mode = configs["label_mode"]
labels = configs["labels"]
image_size = configs["image_size"]


train_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="training",
)

val_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="validation",
)

test_dataset = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    shuffle=False, # do not shuffle for later evaluation, alphanum sort
)

configs.update({"val_steps": val_dataset.samples // configs["batch_size"]})

Found 526 images belonging to 2 classes.
Found 26 images belonging to 2 classes.
Found 33 images belonging to 2 classes.


In [9]:
dropout_rate = 0.2
def define_model(pretrained):
    conv_base = pretrained(
        include_top=False,
        weights=PATHS.resnet50_weights,
        input_shape=(*image_size, 3),
        pooling='avg',
    )
    print("Num trainable at load:", len(conv_base.trainable_weights)) 
    conv_base.trainable = False
    print("Num trainable:", len(conv_base.trainable_weights)) 
    
    x = conv_base.output
    
    # layers at this stage are arbitrary
    # can be subjected to hyperparam tuning
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=512, activation='relu')(x)
    x = keras.layers.Dropout(configs["dropout_rate"])(x)
    outputs = keras.layers.Dense(units=2, activation="softmax")(x)
    model = keras.Model(inputs=conv_base.input, outputs=outputs)
    return model

In [10]:
pretrained = keras.applications.ResNet50
tf.keras.backend.clear_session()
model = define_model(pretrained)

2022-11-29 09:10:02.278385: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1


Num trainable at load: 212
Num trainable: 0


In [11]:
from tensorflow.keras import metrics
thresh = configs["thresh"]
metrics_dict = {
    "ACC":  metrics.BinaryAccuracy(name="ACC", threshold=thresh),
    "AUC-ROC": metrics.AUC(name='ROC', curve='ROC'),
    "AUC-PR": metrics.AUC(name='PR', curve='PR'),
    "TP": metrics.TruePositives(name="TP", thresholds=thresh),
    "TN": metrics.TrueNegatives(name="TN", thresholds=thresh),
    "FP": metrics.FalsePositives(name="FP", thresholds=thresh),
    "FN": metrics.FalseNegatives(name="FN", thresholds=thresh),
}

In [16]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses

# opt = optimizers.Adam(learning_rate=1e-06)
opt = optimizers.Adam()
met = list(metrics_dict.values())

model.compile(
    loss=losses.CategoricalCrossentropy(),
    optimizer=opt,
    metrics=met,
)

In [17]:
# verify arch
# base predictions with untrained classif head
base_preds = model.predict(test_dataset)
base_preds

2022-11-29 07:30:00.000557: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-11-29 07:30:02.283001: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400


array([[0.5667606 , 0.43323943],
       [0.5627947 , 0.43720528],
       [0.5443049 , 0.4556951 ],
       [0.56893605, 0.43106392],
       [0.55799603, 0.4420039 ],
       [0.5635557 , 0.43644425],
       [0.56881726, 0.43118274],
       [0.5747061 , 0.42529386],
       [0.5447857 , 0.45521435],
       [0.5688906 , 0.43110946],
       [0.5668231 , 0.43317688],
       [0.53785646, 0.46214354],
       [0.54906595, 0.45093405],
       [0.56130916, 0.4386908 ],
       [0.5762923 , 0.42370775],
       [0.5654131 , 0.43458688],
       [0.56985587, 0.43014416],
       [0.5654189 , 0.43458113],
       [0.5607474 , 0.43925261],
       [0.5582796 , 0.44172043],
       [0.5663099 , 0.43369007],
       [0.5763818 , 0.42361823],
       [0.57434136, 0.4256586 ],
       [0.5477032 , 0.4522968 ],
       [0.5291214 , 0.47087863],
       [0.57713753, 0.42286247],
       [0.5820543 , 0.41794568],
       [0.5181084 , 0.4818915 ],
       [0.57246435, 0.42753565],
       [0.59290725, 0.40709272],
       [0.

In [18]:
from interpretation import ValLog, GRADCamLogger

# initialize run
run = wandb.init(
    project=PROJECT_NAME, 
    name=RUN_NAME,
    config=configs, 
    job_type='train',
)

wandb_callback = wandb.keras.WandbCallback(
    monitor="val_ROC",
    mode="max",
    save_model=True,
    save_graph=True,
    compute_flops=True,
)

callbacks = [
    wandb_callback,
    ValLog(generator=val_dataset, num_log_batches=1),
    GRADCamLogger(generator=test_dataset, 
                  layer_name=GRADCAM_LAYER_NAME, num_log_batches=1)
    
]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [19]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=configs["epochs"], 
    shuffle=True,
    callbacks=callbacks,
)
run.finish()

2022-11-29 07:30:43.903195: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-11-29 07:30:43.903529: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-11-29 07:30:43.908363: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1
2022-11-29 07:30:43.927382: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.019ms.
  function_optimizer: function_optimizer did nothing. time = 0.003ms.



Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/5
9/9 [==============================] - 18s 1s/step - loss: 0.7050 - ACC: 0.5589 - ROC: 0.5865 - PR: 0.5791 - TP: 294.0000 - TN: 294.0000 - FP: 232.0000 - FN: 232.0000 - val_loss: 0.5855 - val_ACC: 0.6923 - val_ROC: 0.7456 - val_PR: 0.7010 - val_TP: 18.0000 - val_TN: 18.0000 - val_FP: 8.0000 - val_FN: 8.0000


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
2022-11-29 07:31:17.064354: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_073010-2eh4nu83/files/model-best/assets


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_073010-2eh4nu83/files/model-best)... Done. 0.7s


...

/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_073010-2eh4nu83/files/model-best)... Done. 0.7s


Epoch 3/5
9/9 [==============================] - 10s 1s/step - loss: 0.5924 - ACC: 0.6977 - ROC: 0.7532 - PR: 0.7449 - TP: 367.0000 - TN: 367.0000 - FP: 159.0000 - FN: 159.0000 - val_loss: 0.5820 - val_ACC: 0.7692 - val_ROC: 0.7426 - val_PR: 0.6902 - val_TP: 20.0000 - val_TN: 20.0000 - val_FP: 6.0000 - val_FN: 6.0000
Epoch 4/5
9/9 [==============================] - 10s 1s/step - loss: 0.5916 - ACC: 0.7053 - ROC: 0.7464 - PR: 0.7304 - TP: 371.0000 - TN: 371.0000 - FP: 155.0000 - FN: 155.0000 - val_loss: 0.5707 - val_ACC: 0.6923 - val_ROC: 0.7840 - val_PR: 0.7541 - val_TP: 18.0000 - val_TN: 18.0000 - val_FP: 8.0000 - val_FN: 8.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_073010-2eh4nu83/files/model-best/assets


/home/mdorosan/.conda-envs/msds2022-ml3/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221129_073010-2eh4nu83/files/model-best)... Done. 0.7s


Epoch 5/5
9/9 [==============================] - 10s 1s/step - loss: 0.5586 - ACC: 0.7224 - ROC: 0.7824 - PR: 0.7756 - TP: 380.0000 - TN: 380.0000 - FP: 146.0000 - FN: 146.0000 - val_loss: 0.5706 - val_ACC: 0.7308 - val_ROC: 0.7722 - val_PR: 0.7382 - val_TP: 19.0000 - val_TN: 19.0000 - val_FP: 7.0000 - val_FN: 7.0000


ACC,▁▆▇▇█
FN,█▃▂▂▁
FP,█▃▂▂▁
PR,▁▆▇▆█
ROC,▁▆▇▇█
TN,▁▆▇▇█
TP,▁▆▇▇█
epoch,▁▃▅▆█
loss,█▃▃▃▁
val_ACC,▁▄█▁▄
val_FN,█▅▁█▅


In [20]:
run = wandb.init(project=PROJECT_NAME, job_type="inference", name=RUN_NAME)
model_at = run.use_artifact("model-" + RUN_NAME + ":latest")
model_dir = model_at.download()
print("model: ", model_dir)
best_model = keras.models.load_model(model_dir)

wandb: Downloading large artifact model-pretrained_from_radimagenet:latest, 106.02MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:0.3


model:  ./artifacts/model-pretrained_from_radimagenet:v2


In [21]:
# test metrics
import utils

metrics_results = best_model.evaluate(test_dataset)
metrics_results = dict(zip(["loss"] + list(metrics_dict.keys()), 
                           metrics_results))
tp, fp, tn, fn = (metrics_results["TP"], metrics_results["FP"], 
                  metrics_results["TN"], metrics_results["FN"])

add_metrics = {
    "SENSITIVITY": utils.get_sensitivity(tp, fp, tn, fn),
    "SPECIFICTY": utils.get_specificity(tp, fp, tn, fn),
    "PPV": utils.get_ppv(tp, fp, tn, fn),
    "NPV": utils.get_npv(tp, fp, tn, fn),
    "F1" : utils.get_fbeta(tp, fp, tn, fn, beta=1),
}
metrics_results.update(add_metrics)

print(f"Metrics: \n", metrics_results)

columns = list(metrics_results.keys())
metrics_table = wandb.Table(columns=columns)
metrics_table.add_data(*metrics_results.values())
wandb.run.log({METRICS_TABLE_NAME : metrics_table})

# add logging of confusion matrix image from matplotlib
           
# get preds
trained_preds = best_model.predict(test_dataset)
run.finish()

1/1 [==============================] - 3s 3s/step - loss: 0.3641 - ACC: 0.9394 - ROC: 0.9871 - PR: 0.9877 - TP: 31.0000 - TN: 31.0000 - FP: 2.0000 - FN: 2.0000
Metrics: 
 {'loss': 0.3640812635421753, 'ACC': 0.939393937587738, 'AUC-ROC': 0.9871441721916199, 'AUC-PR': 0.9876718521118164, 'TP': 31.0, 'TN': 31.0, 'FP': 2.0, 'FN': 2.0, 'SENSITIVITY': 0.9393939393939394, 'SPECIFICTY': 0.9393939393939394, 'PPV': 0.9393939393939394, 'NPV': 0.9393939393939394, 'F1': 0.9393939393939394}


## End